In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
LinkedIn Selenium Scraper
-------------------------
Ce script permet de scraper des posts LinkedIn en rapport avec l'IA
et d'extraire leurs métriques d'engagement (likes, commentaires, partages)
en utilisant un compte LinkedIn existant.

Développé pour collecter des données pour fine-tuning.
"""

import os
import time
import json
import random
import argparse
import pandas as pd
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException

# Configuration des dossiers
RESULTS_DIR = 'resultats'
os.makedirs(RESULTS_DIR, exist_ok=True)

In [2]:
class LinkedInSeleniumScraper:
    """Classe pour scraper des posts LinkedIn avec Selenium"""

    def __init__(self, headless=False, user_agent=None):
        """
        Initialise le scraper LinkedIn

        Args:
            headless (bool): Mode headless (sans interface graphique)
            user_agent (str): User agent personnalisé
        """
        self.driver = None
        self.headless = headless
        self.user_agent = user_agent or "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36"
        self.base_url = "https://www.linkedin.com"
        self.search_url = "https://www.linkedin.com/search/results/content/"
        self.posts_data = []
        self.is_logged_in = False

        # Paramètres de sécurité
        self.min_delay = 2.0  # Délai minimum entre les actions (secondes)
        self.max_delay = 5.0  # Délai maximum entre les actions (secondes)
        self.scroll_delay = 1.5  # Délai après chaque défilement
        self.max_posts_per_search = 20  # Nombre maximum de posts par recherche
        self.max_searches_per_session = 5  # Nombre maximum de recherches par session

        # Initialiser le navigateur
        self._setup_driver()

In [3]:
def _setup_driver(self):
        """Configure le driver Selenium avec les options appropriées"""
        chrome_options = Options()

        if self.headless:
            chrome_options.add_argument("--headless")

        # Options pour éviter la détection
        chrome_options.add_argument(f"user-agent={self.user_agent}")
        chrome_options.add_argument("--disable-blink-features=AutomationControlled")
        chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
        chrome_options.add_experimental_option("useAutomationExtension", False)

        # Options supplémentaires
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        chrome_options.add_argument("--disable-gpu")
        chrome_options.add_argument("--window-size=1920,1080")

        # Créer le driver
        # Assurez-vous que chromedriver est dans le PATH ou spécifiez le chemin avec Service
        # service = Service('/path/to/chromedriver')
        # self.driver = webdriver.Chrome(service=service, options=chrome_options)
        self.driver = webdriver.Chrome(options=chrome_options) # Simplifié si chromedriver est dans le PATH

        # Modifier les propriétés du navigateur pour éviter la détection
        self.driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

        # Définir un timeout par défaut
        self.driver.implicitly_wait(10)

In [4]:
# Bloc 4 : Méthodes Utilitaires de Temporisation et Simulation

# ---- Ajoute un délai aléatoire entre les actions pour simuler un comportement humain ----
def _random_delay(self):
    """Ajoute un délai aléatoire entre les actions pour simuler un comportement humain"""
    delay = random.uniform(self.min_delay, self.max_delay)
    time.sleep(delay)

# ---- Simule un défilement humain sur la page ----
def _simulate_human_scroll(self, scroll_count=3):
    """Simule un défilement humain sur la page"""
    for _ in range(scroll_count):
        # Défilement aléatoire
        scroll_amount = random.randint(300, 700)
        # La ligne suivante doit être correctement indentée sous le 'for'
        self.driver.execute_script(f"window.scrollBy(0, {scroll_amount});")
        time.sleep(self.scroll_delay)  # Cette ligne aussi

# ---- Simule une saisie humaine avec des délais aléatoires entre les caractères ----
def _type_like_human(self, element, text):
    """Simule une saisie humaine avec des délais aléatoires entre les caractères"""
    for char in text:
        element.send_keys(char)
        time.sleep(random.uniform(0.05, 0.25))  # Cette ligne aussi


In [5]:
def login(self, email, password):
        """
        Se connecte à LinkedIn avec les identifiants fournis

        Args:
            email (str): Email du compte LinkedIn
            password (str): Mot de passe du compte LinkedIn

        Returns:
            bool: True si la connexion a réussi, False sinon
        """
        try:
            print("Connexion à LinkedIn...")
            self.driver.get(f"{self.base_url}/login")
            self._random_delay()

            # Remplir le formulaire de connexion
            email_field = self.driver.find_element(By.ID, "username")
            password_field = self.driver.find_element(By.ID, "password")

            # Simuler une saisie humaine
            self._type_like_human(email_field, email)
            self._random_delay()
            self._type_like_human(password_field, password)
            self._random_delay()

            # Cliquer sur le bouton de connexion
            login_button = self.driver.find_element(By.XPATH, "//button[@type='submit']")
            login_button.click()

            # Attendre que la page d'accueil se charge (ajustez le sélecteur si nécessaire)
            WebDriverWait(self.driver, 30).until(
                EC.presence_of_element_located((By.ID, "global-nav")) # Ou un autre élément stable de la page post-connexion
            )

            print("Connexion réussie!")
            self.is_logged_in = True
            return True

        except Exception as e:
            print(f"Erreur lors de la connexion: {e}")
            # Sauvegarder une capture d'écran peut être utile pour le débogage
            # self.driver.save_screenshot("login_error.png")
            return False

In [6]:
def search_posts(self, keyword, language="fr"):
        """
        Recherche des posts LinkedIn avec le mot-clé spécifié

        Args:
            keyword (str): Mot-clé à rechercher
            language (str): Langue des posts (fr pour français) - Note: L'API de recherche peut changer

        Returns:
            list: Liste des URLs des posts trouvés
        """
        if not self.is_logged_in:
            print("Vous devez être connecté pour effectuer une recherche")
            return []

        try:
            print(f"Recherche de posts avec le mot-clé: {keyword}")

            # Construire l'URL de recherche (simplifiée)
            search_query = keyword.replace(" ", "%20")
            # L'URL de recherche et les paramètres peuvent changer fréquemment sur LinkedIn
            url = f"{self.search_url}?keywords={search_query}&origin=GLOBAL_SEARCH_HEADER&sortBy=relevance"
            # Ajouter des filtres peut nécessiter des ajustements basés sur l'interface actuelle de LinkedIn
            # if language:
            #    url += f"&facetGeoRegion=fr%3A0" # Exemple, à vérifier

            self.driver.get(url)
            self._random_delay()

            # Simuler un comportement humain et charger les posts
            self._simulate_human_scroll(scroll_count=5) # Augmenter le scroll pour plus de résultats

            # Extraire les posts
            post_urls = []
            # Les sélecteurs XPath peuvent devenir obsolètes, à vérifier régulièrement
            post_elements = self.driver.find_elements(By.XPATH, "//div[contains(@class, 'feed-shared-update-v2')] | //li[contains(@class, 'reusable-search__result-container')]") # Ajustement possible du sélecteur

            print(f"Nombre d'éléments trouvés correspondant aux posts potentiels: {len(post_elements)}")

            for post in post_elements[:self.max_posts_per_search]:
                try:
                    # Trouver l'URL du post - S'assurer que le lien contient bien /posts/ ou /feed/update/urn:li:activity:
                    # Il peut y avoir plusieurs liens, on cherche celui du post lui-même
                    post_link_elements = post.find_elements(By.XPATH, ".//a[contains(@href, '/posts/') or contains(@href, '/feed/update/urn:li:activity:')]")
                    post_url = None
                    if post_link_elements:
                         # Souvent le premier lien ou un lien spécifique contient l'URL canonique
                        post_url = post_link_elements[0].get_attribute("href")
                        # Nettoyer l'URL (enlever les query params superflus)
                        if post_url and "?" in post_url:
                            post_url = post_url.split("?")[0]

                    if post_url and ("/posts/" in post_url or "/feed/update/urn:li:activity:" in post_url):
                        if post_url not in post_urls: # Éviter les doublons immédiats
                             post_urls.append(post_url)
                             print(f"Post URL trouvée : {post_url}")


                except (NoSuchElementException, StaleElementReferenceException) as e:
                    print(f"Erreur lors de l'extraction de l'URL d'un post: {e}")
                    continue
                except Exception as e_gen:
                    print(f"Erreur générale lors du traitement d'un élément de post: {e_gen}")
                    continue


            print(f"Trouvé {len(post_urls)} URLs de posts uniques pour le mot-clé '{keyword}'")
            return post_urls

        except TimeoutException:
             print("Timeout lors de la recherche des posts.")
             return []
        except Exception as e:
            print(f"Erreur majeure lors de la recherche: {e}")
            # self.driver.save_screenshot("search_error.png")
            return []

In [7]:
def extract_post_data(self, post_url):
        """
        Extrait les données d'un post LinkedIn

        Args:
            post_url (str): URL du post LinkedIn

        Returns:
            dict: Données du post (texte, auteur, date, métriques d'engagement) ou None si erreur majeure
        """
        if not self.is_logged_in:
            print("Vous devez être connecté pour extraire les données d'un post")
            return None

        try:
            print(f"Extraction des données du post: {post_url}")
            self.driver.get(post_url)
            self._random_delay()

            # Attendre que le contenu principal du post soit chargé
            try:
                WebDriverWait(self.driver, 20).until(
                     EC.presence_of_element_located((By.XPATH, "//main | //div[contains(@class, 'core-rail')]")) # Attendre l'élément principal
                )
            except TimeoutException:
                 print(f"Timeout en attendant le chargement principal du post: {post_url}")
                 # Tenter de continuer ou retourner une erreur
                 # return None # Option: abandonner si la page principale ne charge pas

            # Simuler un comportement humain
            self._simulate_human_scroll(scroll_count=2)

            # Initialiser les données du post
            post_data = {
                "url": post_url,
                "text": "",
                "author": "",
                "date": "",
                "likes": 0,
                "comments": 0,
                "shares": 0, # Les partages sont souvent difficiles à obtenir/non affichés directement
                "timestamp": datetime.now().isoformat(),
                "error": None
            }

            # Extraire le texte du post (plusieurs sélecteurs possibles)
            try:
                # Attendre spécifiquement l'élément de texte
                 post_text_element = WebDriverWait(self.driver, 15).until(
                     EC.presence_of_element_located((
                         By.XPATH,
                         "//div[contains(@class, 'update-components-text')]//span[@dir='ltr'] | " # Nouveau sélecteur potentiel
                         "//div[contains(@class, 'feed-shared-update-v2__description-wrapper')]//span | "
                         "//div[contains(@class, 'attributed-text-segment-list__container')] |" # Autre possibilité
                         "//div[@class='feed-shared-update-v2__commentary']" # Ancien sélecteur
                     ))
                 )
                 post_data["text"] = post_text_element.text.strip()
            except (TimeoutException, NoSuchElementException) as e:
                print(f"Impossible de trouver le texte du post: {e}")
                post_data["error"] = "Text extraction failed"


            # Extraire l'auteur du post
            try:
                 author_element = self.driver.find_element(By.XPATH, "//span[contains(@class, 'update-components-actor__name')]//span[@dir='ltr'] | //a[contains(@class, 'update-components-actor__meta-link')]//span | //span[contains(@class, 'feed-shared-actor__name')]") # Sélecteurs multiples
                 post_data["author"] = author_element.text.strip()
            except NoSuchElementException:
                 print("Impossible de trouver l'auteur du post.")
                 if post_data["error"] is None: post_data["error"] = "Author extraction failed"


            # Extraire la date/heure du post (peut être relatif, ex: "2 h")
            try:
                 # Le lien contient souvent un horodatage plus précis dans l'attribut href ou un span caché
                 date_element = self.driver.find_element(By.XPATH, "//a[contains(@class, 'update-components-actor__sub-description-link')]//span | //span[contains(@class, 'feed-shared-actor__sub-description')]//span[contains(@class, 'visually-hidden')] | //span[contains(@class, 'feed-shared-actor__sub-description')]")
                 post_data["date"] = date_element.text.strip() # Peut nécessiter un parsing plus avancé pour convertir en date absolue
            except NoSuchElementException:
                 print("Impossible de trouver la date du post.")
                 if post_data["error"] is None: post_data["error"] = "Date extraction failed"

            # Extraire les métriques d'engagement
            likes = 0
            comments = 0
            shares = 0 # Moins fiable
            try:
                # Chercher le conteneur des réactions/commentaires
                 social_counts_container = self.driver.find_element(By.XPATH, "//div[contains(@class, 'social-details-social-counts')] | //ul[contains(@class, 'social-details-social-counts')]")

                 # Extraire le nombre de likes/réactions
                 try:
                     likes_element = social_counts_container.find_element(By.XPATH, ".//button[contains(@aria-label, 'reaction')] | .//span[contains(@class,'reactions-count')] | .//li[contains(@class, 'social-details-social-counts__reactions')]//span") # Plusieurs possibilités
                     likes_text = likes_element.text.strip() or likes_element.get_attribute('aria-label') # Parfois dans aria-label
                     likes = self._parse_count(likes_text)
                 except NoSuchElementException:
                      print("Likes non trouvés.")


                 # Extraire le nombre de commentaires
                 try:
                     comments_element = social_counts_container.find_element(By.XPATH, ".//button[contains(@aria-label, 'commentaires')] | .//li[contains(@class, 'social-details-social-counts__comments')]//span | .//a[contains(@href, 'comments')]")
                     comments_text = comments_element.text.strip() or comments_element.get_attribute('aria-label')
                     comments = self._parse_count(comments_text)
                 except NoSuchElementException:
                     print("Commentaires non trouvés.")

                 # Extraire le nombre de partages (si disponible)
                 try:
                     shares_element = social_counts_container.find_element(By.XPATH, ".//button[contains(@aria-label, 'reposts')] | .//li[contains(@class, 'social-details-social-counts__shares')]//span | .//li[contains(@class, 'social-details-social-counts__reposts')]//span") # Reposts sont les nouveaux partages
                     shares_text = shares_element.text.strip() or shares_element.get_attribute('aria-label')
                     shares = self._parse_count(shares_text)
                 except NoSuchElementException:
                     print("Partages/Reposts non trouvés.")


            except NoSuchElementException:
                print("Conteneur des métriques sociales non trouvé.")
                if post_data["error"] is None: post_data["error"] = "Social metrics container not found"


            post_data["likes"] = likes
            post_data["comments"] = comments
            post_data["shares"] = shares # Peut rester 0

            print(f"Données extraites: Likes={likes}, Comments={comments}, Shares={shares}")
            return post_data

        except TimeoutException as e:
             print(f"Timeout lors de l'extraction des données du post {post_url}: {e}")
             return {"url": post_url, "error": f"Timeout: {e}", "timestamp": datetime.now().isoformat()}
        except Exception as e:
            print(f"Erreur majeure lors de l'extraction des données du post {post_url}: {e}")
            # self.driver.save_screenshot(f"extract_error_{post_url.split('/')[-2]}.png")
            return {"url": post_url, "error": f"General extraction error: {e}", "timestamp": datetime.now().isoformat()}

In [8]:
def _parse_count(self, count_text):
        """
        Convertit un texte de compteur en nombre (gère les 'k', 'm', ',', et le texte additionnel).

        Args:
            count_text (str): Texte du compteur (ex: "1,2k réactions", "45 commentaires", "1.5M")

        Returns:
            int: Nombre converti
        """
        if not count_text:
            return 0

        # Extraire la partie numérique au début de la chaîne
        import re
        match = re.match(r"([\d.,]+)\s?([km])?", count_text.lower().replace(',', ''))
        if not match:
             # Essayer de trouver un nombre n'importe où dans la chaîne (moins précis)
             num_match = re.search(r"([\d.,]+)", count_text.replace(',', ''))
             if num_match:
                 try:
                     return int(float(num_match.group(1)))
                 except ValueError:
                      return 0
             else:
                 return 0


        number_part = match.group(1)
        multiplier_char = match.group(2)

        try:
            number = float(number_part)
            if multiplier_char == 'k':
                number *= 1000
            elif multiplier_char == 'm':
                number *= 1000000
            return int(number)
        except ValueError:
            return 0 # Retourne 0 si la conversion échoue

In [9]:
def collect_posts_data(self, keywords, max_posts_total=100):
        """
        Collecte les données de posts LinkedIn pour une liste de mots-clés.

        Args:
            keywords (list): Liste de mots-clés à rechercher.
            max_posts_total (int): Nombre maximum total de posts à collecter sur l'ensemble des mots-clés.

        Returns:
            list: Liste des données de posts collectées.
        """
        if not self.is_logged_in:
            print("Vous devez être connecté pour collecter des données")
            return []

        all_post_urls = []
        search_count = 0
        self.posts_data = [] # Réinitialiser les données collectées pour cet appel

        print(f"Début de la collecte pour les mots-clés: {keywords}")
        print(f"Limite de recherches par session: {self.max_searches_per_session}")
        print(f"Limite de posts par recherche: {self.max_posts_per_search}")
        print(f"Limite totale de posts à extraire: {max_posts_total}")


        # Limiter le nombre de recherches par session
        for keyword in keywords:
            if search_count >= self.max_searches_per_session:
                print(f"Limite de {self.max_searches_per_session} recherches atteinte pour cette session.")
                break

            if len(self.posts_data) >= max_posts_total:
                 print(f"Limite totale de {max_posts_total} posts à extraire atteinte.")
                 break


            print(f"\n--- Recherche {search_count + 1}/{self.max_searches_per_session} --- Mot-clé: '{keyword}' ---")
            # Rechercher des posts pour le mot-clé
            post_urls_found = self.search_posts(keyword) # Utilise self.max_posts_per_search
            all_post_urls.extend(post_urls_found)
            search_count += 1

            # Ajouter un délai plus long entre les recherches pour éviter les blocages
            print(f"Pause après la recherche pour '{keyword}'...")
            time.sleep(random.uniform(8.0, 15.0)) # Délai plus long entre les recherches

        # Dédupliquer les URLs collectées sur l'ensemble des recherches
        unique_post_urls = list(dict.fromkeys(all_post_urls)) # Maintient l'ordre et déduplique
        print(f"\nNombre total d'URLs uniques trouvées après déduplication: {len(unique_post_urls)}")


        # Limiter le nombre total de posts dont on va extraire les données
        post_urls_to_process = unique_post_urls[:min(len(unique_post_urls), max_posts_total)]
        print(f"Nombre d'URLs de posts à traiter (limité par max_posts_total={max_posts_total}): {len(post_urls_to_process)}")


        # Extraire les données pour chaque post unique
        extracted_count = 0
        for i, post_url in enumerate(post_urls_to_process):
             if extracted_count >= max_posts_total:
                 print(f"Limite totale de {max_posts_total} posts extraits atteinte.")
                 break

             print(f"\n--- Extraction {i + 1}/{len(post_urls_to_process)} --- URL: {post_url} ---")
             post_data = self.extract_post_data(post_url)
             if post_data: # Vérifier si l'extraction a retourné quelque chose (pas None)
                 self.posts_data.append(post_data)
                 extracted_count += 1
                 # Ajouter un délai entre l'extraction de chaque post
                 print("Pause après l'extraction...")
                 self._random_delay() # Utilise min_delay et max_delay définis dans __init__


        print(f"\nCollecte terminée. {len(self.posts_data)} posts ont été traités.")
        return self.posts_data

In [10]:
def save_data(self, filename_base="linkedin_posts", format="json"):
    """
    Sauvegarde les données collectées dans un fichier.

    Args:
        filename_base (str): Nom de base pour le fichier de sortie.
        format (str): Format de sortie ('json' ou 'csv').
    """
    if not self.posts_data:
        print("Aucune donnée à sauvegarder.")
        return

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = os.path.join(RESULTS_DIR, f"{filename_base}_{timestamp}.{format}")

    print(f"Sauvegarde des données dans {filename}...")

    try:
        if format == "json":
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(self.posts_data, f, ensure_ascii=False, indent=4)

        elif format == "csv":
            df = pd.DataFrame(self.posts_data)

            # S'assurer que toutes les colonnes attendues existent, même si vides
            expected_columns = ["url", "text", "author", "date", "Likes", "comments", "shares", "timestamp", "error"]
            for col in expected_columns:
                if col not in df.columns:
                    df[col] = None  # Valeur par défaut si la colonne est absente

            # Réorganiser les colonnes
            df = df[expected_columns]
            df.to_csv(filename, index=False, encoding='utf-8-sig')  # utf-8-sig pour Excel

        else:
            print(f"Format de fichier non supporté: {format}. Choisissez 'json' ou 'csv'.")
            return

        print(f"Données sauvegardées avec succès dans {filename} ✅")

    except Exception as e:
        print(f"Erreur lors de la sauvegarde des données : {e}")


In [11]:
def close_driver(self):
        """Ferme le navigateur Selenium"""
        if self.driver:
            try:
                print("Fermeture du navigateur...")
                self.driver.quit()
                self.driver = None
                print("Navigateur fermé.")
            except Exception as e:
                print(f"Erreur lors de la fermeture du navigateur: {e}")

In [12]:
if __name__ == "__main__":
    # Configuration de l’analyse des arguments
    parser = argparse.ArgumentParser(description="Scraper de posts LinkedIn sur l’IA.")
    parser.add_argument("-e", "--email", required=True, help="Email du compte LinkedIn.")
    parser.add_argument("-p", "--password", required=True, help="Mot de passe du compte LinkedIn.")
    parser.add_argument("-k", "--keywords", nargs="+", default=["intelligence artificielle", "machine learning", "deep learning"], help="Mots-clés pour la recherche.")
    parser.add_argument("--headless", action="store_true", help="Exécuter en mode headless (sans interface graphique).")
    parser.add_argument("-n", "--num_posts", type=int, default=10, help="Nombre maximum total de posts à scraper.")
    parser.add_argument("-f", "--format", choices=["json", "csv"], default="csv", help="Format de sortie des données.")
    parser.add_argument("--user_agent", help="User agent personnalisé à utiliser.")

    # ✅ Utilisation de parse_known_args pour éviter les conflits avec Jupyter
    args, unknown = parser.parse_known_args()

    # Créer et exécuter le scraper
    scraper = None
    try:
        print("Initialisation du scraper...")
        scraper = LinkedInSeleniumScraper(
            headless=args.headless,
            user_agent=args.user_agent
        )

        # Connexion
        if scraper.login(args.email, args.password):
            # Collecte des données
            collected_data = scraper.collect_posts_data(args.keywords, max_posts_total=args.num_posts)

            # Sauvegarde des données (si des données ont été collectées)
            if collected_data:
                scraper.save_data(filename_base="linkedin_ia_posts", format=args.format)
            else:
                print("Aucune donnée n’a été collectée ou extraite.")
        else:
            print("Échec de la connexion. Arrêt du script.")

    except Exception as e:
        print(f"❌ Une erreur inattendue est survenue dans le script principal : {e}")

    finally:
        # Assurer la fermeture du driver même en cas d’erreur
        if scraper:
            scraper.close_driver()


usage: ipykernel_launcher.py [-h] -e EMAIL -p PASSWORD
                             [-k KEYWORDS [KEYWORDS ...]] [--headless]
                             [-n NUM_POSTS] [-f {json,csv}]
                             [--user_agent USER_AGENT]
ipykernel_launcher.py: error: argument -f/--format: invalid choice: '/Users/Nicolas/Library/Jupyter/runtime/kernel-v34e57c1712cc9431b72d40004cf76315143e531ae.json' (choose from 'json', 'csv')


AttributeError: 'tuple' object has no attribute 'tb_frame'

In [19]:
import sys
sys.argv = [
    "",  # nom fictif du script
    "-e", "monemail@exemple.com",
    "-p", "monmotdepasse",
    "-k", "intelligence artificielle", "IA", "ChatGPT",
    "--num_posts", "10",
    "--format", "csv",
    "--headless"
]


In [20]:
if __name__ == "__main__":
    # Configuration de l’analyse des arguments
    parser = argparse.ArgumentParser(description="Scraper de posts LinkedIn sur l’IA.")
    parser.add_argument("-e", "--email", required=True, help="Email du compte LinkedIn.")
    parser.add_argument("-p", "--password", required=True, help="Mot de passe du compte LinkedIn.")
    parser.add_argument("-k", "--keywords", nargs="+", default=["intelligence artificielle", "machine learning", "deep learning"], help="Mots-clés pour la recherche.")
    parser.add_argument("--headless", action="store_true", help="Exécuter en mode headless (sans interface graphique).")
    parser.add_argument("-n", "--num_posts", type=int, default=10, help="Nombre maximum total de posts à scraper.")
    
    # 🔥 On évite le conflit avec le `-f` de Jupyter en supprimant l'alias court
    parser.add_argument("--format", choices=["json", "csv"], default="csv", help="Format de sortie des données.")

    parser.add_argument("--user_agent", help="User agent personnalisé à utiliser.")

    # ✅ parse_known_args pour ne pas planter avec les arguments système de Jupyter
    args, unknown = parser.parse_known_args()

    # Créer et exécuter le scraper
    scraper = None
    try:
        print("Initialisation du scraper...")
        scraper = LinkedInSeleniumScraper(
            headless=args.headless,
            user_agent=args.user_agent
        )

        # Connexion
        if scraper.login(args.email, args.password):
            print("Connexion réussie.")
            collected_data = scraper.collect_posts_data(args.keywords, max_posts_total=args.num_posts)

            if collected_data:
                scraper.save_data(filename_base="linkedin_ia_posts", format=args.format)
            else:
                print("Aucune donnée n’a été collectée ou extraite.")
        else:
            print("Échec de la connexion. Arrêt du script.")

    except Exception as e:
        print(f"❌ Une erreur inattendue est survenue dans le script principal : {e}")

    finally:
        if scraper:
            scraper.close_driver()


Initialisation du scraper...
❌ Une erreur inattendue est survenue dans le script principal : 'LinkedInSeleniumScraper' object has no attribute '_setup_driver'
